In [ ]:

import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np

from sklearn.neighbors import LocalOutlierFactor


df = pd.read_csv('./REF_Buchungen_2018.csv',sep=';',header=None, encoding='ISO-8859-1', low_memory=False) 

#df.rename({0: 'BereichsId', 1: 'KtoNr', 2:'Datum', 3:'BU_Schl', 4:'GKtoNr', 11:'Umsatz_S', 12:'Umsatz_H'}, axis=1, inplace=True) 
df.rename({0: 'KtoNr', 3:'Datum',6:'Umsatz_S', 7:'Umsatz_H'}, axis=1, inplace=True) 
 
df= df[['KtoNr', 'Datum', 'Umsatz_S', 'Umsatz_H']]

df['Umsatz_H'] = df['Umsatz_H'].str.replace(',','.')
df['Umsatz_H'] = pd.to_numeric(df['Umsatz_H'])

df['Umsatz_S'] = df['Umsatz_S'].str.replace(',','.')
df['Umsatz_S'] = pd.to_numeric(df['Umsatz_S'])
 


df_Umsaetze = df[((df['KtoNr'] >= 8000) & (df['KtoNr'] <= 8589)) | ((df['KtoNr'] >= 8900) & (df['KtoNr'] <= 8919)) | ((df['KtoNr'] >= 8940) & (df['KtoNr'] <= 8959)) |
      ((df['KtoNr'] >= 8700) & (df['KtoNr'] <= 8799))].copy()



df_Umsaetze = df_Umsaetze.drop(columns=['KtoNr'])
df_Umsaetze.head()



In [ ]:
df_Umsaetze['Datum'] = pd.to_datetime(df_Umsaetze['Datum'],format='%d.%m.%Y')
df_Umsaetze.sort_values(by=['Datum'], inplace=True)

# Saldo berechnen
df_Umsaetze['Saldo'] = df.apply(lambda row: row.Umsatz_H - row.Umsatz_S, axis = 1) 
df_Umsaetze = df_Umsaetze.drop(columns=['Umsatz_S','Umsatz_H'])
df_Umsaetze.info()

In [ ]:
#Sortieren der Zahlen nach Datum
df_Umsaetze = df_Umsaetze.set_index(pd.DatetimeIndex(df_Umsaetze['Datum']))
df_Umsaetze = df_Umsaetze['Saldo'].resample("D").sum()

print(df_Umsaetze.values[0:10])
df_Umsaetze.plot(style='k.')


In [ ]:
clf = LocalOutlierFactor(n_neighbors=20)

#Nötig für 1D Values
X = df_Umsaetze.values.reshape(-1,1)
pred = clf.fit_predict(X)

pred[0:10]




In [ ]:
X_scores = clf.negative_outlier_factor_

X_scores[0:20]

In [ ]:
df_model = pd.DataFrame(df_Umsaetze.values, columns = ["umsaetze"])

df_model['scores'] = X_scores
df_model['anomaly'] = pred
df_model.head(20)


In [ ]:
df_model.plot(style='k.', x='Umsaetze', y='Score')